In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import pandas as pd
import data
from IPython.display import HTML, Markdown
# import myWidgets
import logisticRegression
import numpy as np
from sklearn.metrics import confusion_matrix, precision_recall_curve, f1_score, roc_auc_score, recall_score, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import Model
import Trackers.ModelTracker as ModelTracker
import plot_tools
import writing_tools
import FeatureTools
import Trackers.FeatureTracker as FeatureTracker
import Trackers.DataTracker as DataTracker


In [3]:
df = data.get_data()
model_tracker = ModelTracker.ModelTracker()
feature_tracker = FeatureTracker.FeatureTracker(df.copy())
data_tracker = DataTracker.X_tracker()
cols_to_scale = []

$$\text{logit } p = \ln \frac{p}{1-p} = \beta_0 + \beta_1x_1 + ... + \beta_nx_n = z$$
$$ e^z = \frac{p}{1-p} \iff p = \frac{e^z}{1 + e^z} $$
$$p = P(\text{CARAVAN} = 1 | X = x)$$

$$\text{odds } = \frac{p}{1-p}$$

$$OR = e^{\beta}$$

Le odds-ratio permet de mesurer l'effet d'une variable sur la probabilité qu'un évenements se produise (CARAVAN = 1).

Il faut que je calcul l'intervalle de confiance pour assumer qu'une variable est significative selon la valeur de son $OR$.

Pour calculer un intervalle de confiance, j'ai besoin de l'écart-type des variables. Mais puisque OR dépend des rélations entres les variables, je dois utiliser une matrice de covariance, qui est l'inverse de la matrice de Fisher Information.

**Fisher Information :** 

$$\theta = \mathbb{E} \left[ \left( \frac{\partial}{\partial \theta} \log f(X; \theta) \right)^2 \right]$$

$\mathbb{E} = $  espérance de $X$ \
$f(X; \theta) = $ fonction de densité de $X$

Forme matrice : 

$$p_i = P(\text{CARAVAN} = 1 | X_i) = \sigma(X_i \cdot \beta) = \dfrac{e^{X_i \cdot \beta}}{1 + e^{X_i \cdot \beta}} \\
\ell(\beta) = \sum^n_{i=1} \log f(X_i; \theta) =  \sum_{i=1}^{n} \Big[ y_i \log(p_i) + (1 - y_i) \log(1 - p_i) \Big] = - \text{cost function } \\ 
(y_i = \text{la valeur observée du target (0 ou 1))} 
$$
$$
\frac{\partial \ell}{\partial \beta} = X^T (y - p) = \text{gradient ascent} \\
\frac{\partial^2 \ell}{\partial \beta\, \partial \beta^T} = -X^T W X = \text{variation du gradient ascent}
$$

$$\text y_i \sim Bernoulli(p) \iff Var(y_i) = p_i(1 - p_i)$$

$$ W = diag(p_i(1 - p_i)) = Var(Y) =
\begin{bmatrix}
p_1(1-p_1) & 0 & \cdots & 0 \\
0 & p_2(1-p_2) & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & p_n(1-p_n)
\end{bmatrix}$$

Sa calcul la précision de l'estimation d'un coefficient 


In [4]:
categorical_non_ordinales = ['MOSTYPE', 'MOSHOOFD']
df_encoded = pd.get_dummies(df, columns=categorical_non_ordinales, prefix=categorical_non_ordinales, dtype=int, drop_first=True)
X_base_one_hot = df_encoded.copy()
featureTester = FeatureTracker.FeatureTracker(X_base_one_hot)
X_train_np, y_train_np, X_val_np, y_val_np = featureTester.return_split_train_eval(toNpy=True)

model = featureTester.get_trained_model()

AUC         : 0.7951
Precision   : 0.1697
Recall      : 0.5286
F1          : 0.2569
Threshold   : 0.1000
[[914 181]
 [ 33  37]]


In [5]:
coeff = model.w
bias = model.b
odds_ratio = np.exp(coeff)
confiance = 0.95

def fisher_info(X, w, b):
    z = X @ w + b
    p = logisticRegression.sigmoid(z)
    W = np.diag(p * (1-p))
    fi = X.T @ W @ X
    return fi

fi = fisher_info(X_train_np, coeff, bias)
print(np.linalg.det(fi))

4.753692681997185e-11


La matrice de covariance est proche d'être singuliaire, car $\text{det}(\text{fisher\_information}) \approx 0$, elle est presque non-inversible. \
Si je l'inverse, il y a des variances gigantesque et positive/négative.

- Si on regarde qu'elle variance $\lt$ 0, on trouve que 14 d'entres elles sont négatives (elles sont aussi gigantesques) $( Var(y) \lt -10^{14})$. \
29 sont des variables catégoriques non-ordinales à propos du type de cients et 2 sont à propos du nombre et la contribution d'assurance surfboard. \

- Si on regarde ceux qui sont géante, on trouve qu'il y en a 32 d'entres elles qui sont gigantesques.


C'est un signe que ces variables sont peut-être complètement séparer des données, donc pourait prédire constamment le même résultat, ce qui indique que leur $\beta$ tend vers $\pm \infin$.

In [6]:
cov_matrix = np.linalg.inv(fi)
#std = np.sqrt(np.diag(cov_matrix))
X_train, y_train, X_val, y_val = featureTester.return_split_train_eval()
coef_df = pd.DataFrame({
    'feature': X_train.columns,
    'var': np.diag(cov_matrix),
})

print('négatif count in variance:               ', coef_df[coef_df['var'] < 0].shape[0])
print('négatif gigantesque count in variance:   ', coef_df[coef_df['var'] <= -10**13].shape[0])
print('Gigantesque count in variance:           ', coef_df[coef_df['var'] >= 10**(14)].shape[0])

huge_neg_coeff_cols = coef_df[coef_df['var'] < 0]['feature'].values
huge_pos_coeff_cols = coef_df[coef_df['var'] >= 10**(14)]['feature'].values

inf_coef_cols = np.concatenate((huge_neg_coeff_cols, huge_pos_coeff_cols), axis=0)
X_train_inf_coef = X_train[inf_coef_cols].copy()

négatif count in variance:                14
négatif gigantesque count in variance:    14
Gigantesque count in variance:            32


Si on vérifie, on trouve qu'il y a quelque une variable qui prédit toujours CARAVAN = 0 et le reste sont des cas très rares.

In [7]:
huge_neg_coeff_cols = coef_df[coef_df['var'] < 0]['feature'].values
huge_pos_coeff_cols = coef_df[coef_df['var'] >= 10**(14)]['feature'].values

inf_coef_cols = np.concatenate((huge_neg_coeff_cols, huge_pos_coeff_cols), axis=0)
X_train_inf_coef = X_train[inf_coef_cols].copy()

cols_with_zeros_targets = []
cols_with_all_targets = []
cols_with_rare_outcomes = {}

for c in inf_coef_cols:
    crossTab = pd.crosstab(X_train_inf_coef[c], y_train)

    if 1 in crossTab.index:
        #print(f'\n{c} = 1')
        not_target_count = crossTab.loc[1, 0]
        target_count = crossTab.loc[1, 1]
        #print(f"    (CARAVAN=0) : {not_target_count}")

        if target_count == 0:
            #print(f'    Prédit toujours CARAVAN=0')
            cols_with_zeros_targets.append(c)
        elif not_target_count == 0:
            #print(f'    Prédit toujours CARAVAN=1')
            cols_with_all_targets.append(c)
        else:
           #print(f"    (CARAVAN=1): {target_count}")
           cols_with_rare_outcomes[c] = np.array([target_count, not_target_count])
    else:
        print('what?')

Les variables qui prédisent tous le temps CARAVAN = 0 dans le training set sont inutiles donc je vais les retirer.

In [8]:
featureTester.remove_list(cols_with_zeros_targets)
X = featureTester.flush_to_df()

X_train_np, y_train_np, X_val_np, y_val_np = featureTester.return_split_train_eval(toNpy=True)
model = Model.create_model(X_train_np, y_train_np, X_val_np, y_val_np, learning_rate=0.01, set_threshold_to=0.1)

Les autres variables ne sont pas complètement séparer, mais peut-etre souffre de multicolinéarité, ce qui serait la cause de leur variance gigantesque.

Précédemment, j'avais utilisé une matrice de corrélation pour trouver les corrélations entre variables, mais le problème est qu'elle évalue seulement la colinéarité entre deux variables. Pour ce problème, c'est mieux d'utiliser le VIF, car il permet d'analyser l'impact de tout les autres variables sur la variance d'un coefficient, ce qui est exactement ce que j'ai besoin pour trouver la cause de la variance énorme des variables restantes. 

$$VIF = \dfrac{1}{1-R^2}$$
$$
X_i = \beta_0 + \sum_{j \neq i}(\beta_j X_j) + \varepsilon_i \\
\hat{\beta} = (X^TX)^{-1}X^Ty 
$$
$$\varepsilon_i = X_i -\hat{X_i} \iff Var(\varepsilon_i) = Var(X_i) -Var(\hat{X_i})$$
$$\sum{\varepsilon_i} = \sum{(X_i -\hat{X_i})} = 0$$
$$
R^{2}_{i} = \frac{Var(\hat{X_i})}{Var(X_i)} = \frac{ Var({X_i}) - Var(\varepsilon_i)}{Var(X_i)} 
= 1 - \frac{Var(\varepsilon_i)}{Var({X_i})}
$$

In [9]:
def ols(X, y):
    w = np.linalg.lstsq(X, y, rcond=None)[0]
    return w

def regressionLineaire(X, y):
    w = ols(X, y)
    residual = y - X @ w
    return w, residual

def auxRegLin(X, eps, tol=1e-12):
    var_residual = np.sum(eps**2)
    xm = np.mean(X)
    var_X = np.sum((X - xm)**2)
    if var_X <= tol: return 0
    return (1 - var_residual/var_X)
    
def vif(X, cols):
    vif_res = {}
    for c in cols:
        X_i = X[c].to_numpy()
        X_hat_i = X.drop(c, axis=1).to_numpy()
        X_hat = np.column_stack([np.ones(len(X_i)), X_hat_i])
        w, residu = regressionLineaire(X_hat, X_i)
        R2 = auxRegLin(X_i, residu)
        vif = np.inf if np.isclose(1 - R2, 0) else 1 / (1 - R2)
        vif_res[c] = vif
    return pd.DataFrame.from_dict(vif_res, orient='index', columns=['VIF']).sort_values(by='VIF', ascending=False)

df_vif = vif(X.drop('CARAVAN', axis=1), cols_with_rare_outcomes)


In [10]:
index_mid_corr = df_vif[(df_vif['VIF'] > 1) & (df_vif['VIF'] <= 5)].index
index_high_corr = df_vif[(df_vif['VIF'] > 5) & (df_vif['VIF'] <= 10)].index
index_serious_corr = df_vif[(df_vif['VIF'] > 10)].index

Tout les variables avec un $R^2$ = $\inf$ (colinéarité parfaite, donc parfaitement prédite par les autres variables) sont des variables de types MOSTYPE (customer sub-type) ou MOSHOOFD (customer main type). Je vais les retirer puisqu'elle n'aide pas le modèle. Les seules que je vais garder sont eux avec 1 < VIF < 5. Plus grand leur résultat VIF indique qu'ils sont hautement corrélés.

Les métriques restent similaires à avant le retrait de ces variables, donc c'est une confirmation que le retrait n'a pas tellement affecté les prédictions du modèles.

In [11]:
featureTester.remove_list(index_high_corr)
featureTester.remove_list(index_serious_corr)

X = featureTester.flush_to_df()

model = featureTester.get_trained_model()

AUC         : 0.7944
Precision   : 0.1689
Recall      : 0.5286
F1          : 0.2561
Threshold   : 0.1000
[[913 182]
 [ 33  37]]


Maintenant, on retourne au calcul de Fisher information pour trouver les variables les plus utiles pour prédire CARAVAN=1

In [12]:
X_train_np, y_train_np, X_val_np, y_val_np = featureTester.return_split_train_eval(toNpy=True)
fi = FeatureTools.fisher_info(X_train_np, model.w, model.b)
print(np.linalg.det(fi))

-1.615584458150003e+94


In [13]:
eigen_values = np.linalg.eigvals(fi)
print('Nombre de eigen values négative: ', (eigen_values<0).sum())

Nombre de eigen values négative:  1


In [14]:
eigen_values, eigen_vectors = np.linalg.eig(fi)
negative_values = (eigen_values<0)
idx_problem = [i for i, is_negative in enumerate(negative_values) if is_negative]

problematic_vector = eigen_vectors[:, idx_problem]

index = X.drop('CARAVAN', axis=1).columns

problematic_variables_df = pd.DataFrame(np.abs(problematic_vector), index=index)
Markdown(problematic_variables_df.sort_values(by=0, ascending=False).head().to_markdown())


|         |           0 |
|:--------|------------:|
| PZEILPL | 0.707107    |
| AZEILPL | 0.707107    |
| AVRAAUT | 5.81657e-15 |
| AGEZONG | 4.81223e-15 |
| AWAOREG | 2.64977e-15 |

Seulement une personne dans le training set a PZEILPL ou AZEILPL > 0 et CARAVAN = 1

In [15]:
print(FeatureTools.get_target_count_of_variables(X_train, y_train, ['PZEILPL', 'AZEILPL']))


|    |   PZEILPL (CARAVAN=1) |   AZEILPL (CARAVAN=1) |
|---:|----------------------:|----------------------:|
|  0 |                   277 |                   277 |
|  1 |                     1 |                     1 |


Parfait, maintenant le determinant n'est plus négatif

In [16]:
featureTester.remove_list(['PZEILPL', 'AZEILPL'])
X = featureTester.flush_to_df()
model = featureTester.get_trained_model(print_stats=False)

X_train_np, y_train_np, X_val_np, y_val_np = featureTester.return_split_train_eval(toNpy=True)
fi = FeatureTools.fisher_info(X_train_np, model.w, model.b)
print(np.linalg.det(fi))

9.848675513629938e+128


In [17]:
import scipy.stats as st

coeff = model.w 
bias = model.b 
odds_ratio = np.exp(coeff) 
confiance = 0.95

X_train_np, y_train_np, X_val_np, y_val_np = featureTester.return_split_train_eval(toNpy=True)

fi = FeatureTools.fisher_info(X_train_np, model.w, model.b)
cov_matrix = np.linalg.inv(fi)
std = np.sqrt(np.diag(cov_matrix))

surface = 1 - (1-confiance)/2
z = st.norm.ppf(surface)

bi = coeff - z * std
bs = coeff + z * std

bi_or = np.exp(bi)
bs_or = np.exp(bs)
    
coef_df = pd.DataFrame({
    'feature': X.drop('CARAVAN', axis=1).columns,
    '$\\beta_n$': model.w,
    '$OR$': odds_ratio,
    '$bi_{OR}$': bi_or,
    '$bs_{OR}$': bs_or
})

coef_df = coef_df.round(4)

mask_signif = (coef_df['$bi_{OR}$'] > 1) | (coef_df['$bs_{OR}$'] < 1)
Markdown(coef_df[mask_signif].to_markdown())

|    | feature   |   $\beta_n$ |   $OR$ |   $bi_{OR}$ |   $bs_{OR}$ |
|---:|:----------|------------:|-------:|------------:|------------:|
| 44 | PPERSAUT  |      0.1974 | 1.2182 |      1.1104 |      1.3365 |

$$OR > 1 \to \text{augmente les chances que CARAVAN = 1}$$
$$OR < 1 \to \text{diminue les chances que CARAVAN = 1}$$
$$OR = 1 \to \text{aucun effet sur les chances que CARAVAN = 1}$$

Présentement, voici à quoi ressemble le modèle sans aucune variable additionnel.

In [18]:
DATA_V4 = 'DATA_FE2_NO_ADDED_VARS_JUST_FILTERED_BASE'
X = featureTester.flush_to_df()
data_tracker.add(DATA_V4, X)

Dataset DATA_FE2_NO_ADDED_VARS_JUST_FILTERED_BASE saved.



In [19]:
X_train, y_train, X_val, y_val = featureTester.return_split_train_eval()
cols = X_train.columns

df_vif = FeatureTools.vif(X_train, cols)
cols = df_vif[df_vif['VIF'] >= 10].index
cols.sort_values()

Index(['AAANHANG', 'ABROM', 'ABYSTAND', 'AGEZONG', 'AMOTSCO', 'AVRAAUT',
       'AWALAND', 'AWAOREG', 'AWAPART', 'AWERKT', 'MAUT0', 'MAUT1', 'MFALLEEN',
       'MFGEKIND', 'MFWEKIND', 'MGODPR', 'MHHUUR', 'MHKOOP', 'MINK4575',
       'MINKM30', 'MOPLHOOG', 'MOPLLAAG', 'MOPLMIDD', 'MRELGE', 'MRELOV',
       'MSKC', 'MZFONDS', 'MZPART', 'PAANHANG', 'PBROM', 'PBYSTAND', 'PGEZONG',
       'PMOTSCO', 'PTRACTOR', 'PVRAAUT', 'PWALAND', 'PWAOREG', 'PWAPART'],
      dtype='object')

Tout les variables qui débute par A sont des variables qui indique le nombre de police d'assurance x.
Je vais tout les regrouper en une seule variable 'total' puisqu'elle mesure la meme chose que les variables P mais sous une différentes unité. (Nombre de polices vs Contribution). 

| Type d'assurance                          | Variables associées | Description complète |
|------------------------------------------|-------------------|--------------------|
| Assurance vie / accidents / santé         | ALEVEN, APERSONG, AGEZONG, AWAOREG, ABRAND, ABYSTAND | Number of life insurances, private accident insurance policies, family accidents insurance policies, disability insurance policies, fire policies, social security insurance policies |
| Third party insurance                     | AWAPART, AWABEDR, AWALAND | Private third party insurance, third party insurance (firms), third party insurance (agriculture) |
| Assurance véhicule quotidien              | APERSAUT, AMOTSCO, ABROM | Number of car policies, motorcycle/scooter policies, moped policies |
| Assurance véhicule de travail / poids lourd | AVRAAUT, ATRACTOR, AWERKT, AAANHANG, ABESAUT | Number of lorry, tractor, agricultural machines, trailer policies, delivery van policies |
| Assurance propriété                       | AINBOED           | Number of property insurance policies |
| Assurance véhicule de loisir              | APLEZIER, AFIETS, AZEILPL | Number of boat, bicycle, surfboard policies |





In [20]:
print('|----- Avant le regroupement -----|')
model = featureTester.get_trained_model()

|----- Avant le regroupement -----|
AUC         : 0.7944
Precision   : 0.1689
Recall      : 0.5286
F1          : 0.2561
Threshold   : 0.1000
[[913 182]
 [ 33  37]]


In [21]:
featureTester.restore('AZEILPL') # car retirer précédemment
X = featureTester.flush_to_df()

life_accidents_health = np.array(['ALEVEN','APERSONG','AGEZONG','AWAOREG','ABRAND','ABYSTAND'])
third_party_insurance = np.array(['AWAPART','AWABEDR','AWALAND'])
daily_vehicle = np.array(['APERSAUT','AMOTSCO','ABROM'])
work_heavy_vehicle = np.array(['AVRAAUT','ATRACTOR','AWERKT','AAANHANG','ABESAUT'])
property_insurance = np.array(['AINBOED'])
leisure_vehicle = np.array(['APLEZIER','AFIETS','AZEILPL'])

all_policies = np.concatenate([life_accidents_health, third_party_insurance, daily_vehicle, work_heavy_vehicle, property_insurance, leisure_vehicle])

featureTester.add('total_policies', X[all_policies].sum(axis=1), toScale=True)
featureTester.remove_list(all_policies)
X = featureTester.flush_to_df()

print('|----- Après le regroupement -----|')
model = featureTester.get_trained_model()

|----- Après le regroupement -----|
AUC         : 0.7943
Precision   : 0.1735
Recall      : 0.5429
F1          : 0.2630
Threshold   : 0.1000
[[914 181]
 [ 32  38]]


In [22]:
X_train, y_train, X_val, y_val = featureTester.return_split_train_eval()
cols = X_train.columns

df_vif = FeatureTools.vif(X_train, cols)
df_vif[df_vif['VIF'] >= 10]

,VIF
MHHUUR,1607.413278
MHKOOP,1605.011609
MZFONDS,802.201002
MZPART,801.486528
MOPLLAAG,31.976265
MRELGE,22.988768
MFWEKIND,21.727982
MRELOV,18.949109
MOPLMIDD,18.107363
MFALLEEN,15.636070


(Rented house) MHHUUR $\iff$ MHKOOP (Home owner) \
(National Health Service) MZFONDS $\iff$ MZPART (Private health insurance)

MOPLLAAG - MOPLMIDD - MOPLHOOG (retire une, devient catégorie de référence)

MRELGE - MRELOV - MRELSA (retire une, devient catégorie de référence) (MRELOV, car plus facile a interpreter apres, marier ou habite ensemble ou sinon catégorie de référence aka other relation)

MFWEKIND - MFGEKIND - MFALLEEN (meme chose que ^) (MFGEKIND)

MAUT0 - MAUT1 - MAUT2 (mm chose) (MAUT0)

MINK* (mm chose) (MINKM30)

MGOD* (mm chose) (MGODOV)

MBER* (mm chose) (MBERARBO)

In [23]:
featureTester.remove_list(['MHHUUR', 'MZFONDS', 'MOPLLAAG', 'MRELOV', 'MFGEKIND', 'MAUT0', 'MINKM30', 'MGODOV', 'MBERARBO'])
X = featureTester.flush_to_df()

In [24]:
X_train, y_train, X_val, y_val = featureTester.return_split_train_eval()
cols = X_train.columns

df_vif = FeatureTools.vif(X_train, cols)
print(df_vif[df_vif['VIF'] >= 5].to_markdown())

|                |     VIF |
|:---------------|--------:|
| total_policies | 9.87819 |
| MSKA           | 8.15507 |
| MSKC           | 7.76223 |


Pour le reste des variables, je vais utiliser le score Z pour voir si elle sont significative ou pas.

$$H_0 : w_i = 0\\
H_1 : w_i \neq 0 
$$

In [25]:
X_train, y_train, X_val, y_val = featureTester.return_split_train_eval()
model = featureTester.get_trained_model(print_stats=False)

Elles contiennent tous $0$, il y a la possibilité que l'effet de ces variables soit nul.

In [26]:
fi = FeatureTools.fisher_info(X_train.to_numpy(), model.w, model.b)
cov_matrix = np.linalg.inv(fi)
std = np.sqrt(np.diag(cov_matrix))

z_scores = model.w / std

z_scores_df = pd.DataFrame({
    'Variable': X_train.columns,
    '$w_i$': model.w,
    r'$\sigma$': std,
    '$Z$': abs(z_scores)
})

z_scores_df['$b_i$'] = z_scores_df['$w_i$'] - 1.96 * z_scores_df[r'$\sigma$']
z_scores_df['$b_s$'] = z_scores_df['$w_i$'] + 1.96 * z_scores_df[r'$\sigma$']

vars = ['total_policies', 'MSKA', 'MSKC']

Markdown(z_scores_df[z_scores_df['Variable'].isin(vars)].to_markdown())

|    | Variable       |      $w_i$ |   $\sigma$ |      $Z$ |     $b_i$ |     $b_s$ |
|---:|:---------------|-----------:|-----------:|---------:|----------:|----------:|
| 17 | MSKA           | -0.0337001 |  0.0982806 | 0.342896 | -0.22633  | 0.15893   |
| 20 | MSKC           | -0.109832  |  0.0758362 | 1.44828  | -0.258471 | 0.0388066 |
| 60 | total_policies |  0.102733  |  0.169785  | 0.605078 | -0.230046 | 0.435512  |

In [27]:
X_train, y_train, X_val, y_val = featureTester.return_split_train_eval()
cols = X_train.columns

df_vif = FeatureTools.vif(X_train, cols, return_coef=True)
Markdown(df_vif[df_vif['Target'] == 'total_policies'].sort_values(by='coef', key=abs, ascending=False).head().to_markdown())

|      | Target         | Variable   |      coef |
|-----:|:---------------|:-----------|----------:|
| 3660 | total_policies | Intercept  | -1.21704  |
| 3710 | total_policies | PFIETS     |  0.882518 |
| 3700 | total_policies | PAANHANG   |  0.517393 |
| 3693 | total_policies | PWAPART    |  0.410282 |
| 3702 | total_policies | PWERKT     |  0.388062 |

In [28]:
vars = ['total_policies', 'MSKA', 'MSKC']
featureTester.remove_list(vars)
X = featureTester.flush_to_df()

X_train, y_train, X_val, y_val = featureTester.return_split_train_eval()
cols = X_train.columns

df_vif = FeatureTools.vif(X_train, cols)
print(df_vif[df_vif['VIF'] >= 5].to_markdown())

| VIF   |
|-------|


In [29]:
model = featureTester.get_trained_model(print_stats=False)

In [30]:
Markdown(FeatureTools.or_with_ic(model, X_train.to_numpy(), X_train.columns))

|    | feature   |   $\beta_n$ |   $OR$ |   $bi_{OR}$ |   $bs_{OR}$ |
|---:|:----------|------------:|-------:|------------:|------------:|
| 33 | PPERSAUT  |      0.1848 | 1.203  |      1.1456 |      1.2633 |
| 45 | PBRAND    |      0.0925 | 1.0969 |      1.0076 |      1.194  |
| 16 | MBERARBG  |     -0.1017 | 0.9033 |      0.8182 |      0.9973 |
|  4 | MGODPR    |     -0.1219 | 0.8852 |      0.7894 |      0.9928 |
|  8 | MFALLEEN  |     -0.1654 | 0.8476 |      0.7584 |      0.9472 |
|  5 | MGODGE    |     -0.1766 | 0.8382 |      0.7456 |      0.9423 |

In [31]:
model = featureTester.get_trained_model()

AUC         : 0.7715
Precision   : 0.1796
Recall      : 0.5286
F1          : 0.2681
Threshold   : 0.1000
[[926 169]
 [ 33  37]]
